In [1]:
from typing import Dict, List
from collections import defaultdict
import random

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from autocorrect import Speller
from pymorphy2 import MorphAnalyzer
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Читаем данные и выполняем обзор имеющихся данных

In [2]:
df_train = pd.read_csv('train_dataset_train.csv')
df_test = pd.read_csv('test_dataset_test.csv')

In [3]:
df_train.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
0,2246,Помогите начальник Льговского рэс не реагирует...,"Нарушения, связанные с содержанием электросети...",Администрация Льговского района,3
1,380,<p>По фасаду дома по адресу ул. Урицкого 22 пр...,Аварийные деревья,Администрация города Курска,3
2,2240,Агресивные собаки. На радуге там стая из подро...,Безнадзорные животные,Администрация города Курска,1
3,596,<p>На пересечении &nbsp;улиц Сосновская и Бере...,Нескошенная сорная растительность в местах общ...,Комитет дорожного хозяйства города Курска,3
4,1797,<p style=`text-align:justify;`><span style=`ba...,Аварийные деревья,Комитет городского хозяйства города Курска,3


In [4]:
df_test.head()

,id,Текст Сообщения,Тематика,Ответственное лицо
0,843,<p>Здравствуйте. На улице Мира &nbsp;было заме...,Неработающее наружное освещение,Администрация Курчатовского района
1,1422,<p>Уже вторую неделю не горит уличное освещени...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
2,2782,Не работает освещение во дворе дома 11а по Эне...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
3,2704,После покоса сорной растительности на газоне м...,Неудовлетворительная уборка улиц и тротуаров,Администрация Центрального округа города Курска
4,1,<p>Прошу принять меры к водителю маршрута 263:...,Неудовлетворительный внешний вид (поведение) в...,Администрация города Курска


## Подготовка и преобразование данных

In [169]:
# Функции для обработки текста

RU_LETTERS = re.compile('[а-яё]+')
CLEAN_HTML = re.compile('<.*?>')
spell = Speller('ru')
morph = MorphAnalyzer()

# Удяляем html-теги, в нижний регистр, оставляем только русские буквы. Разбиваем на токены.
# Удаляем слова короче 3-х символов как неинформативные.
def format_text(raw_text: str) -> str:
    # Убираем html-теги
    txt = re.sub(CLEAN_HTML, '', raw_text)
    
    # Приводим к нижнему регистру
    txt = txt.lower()
    
    # Оставляем только слова из русских букв
    txt = " ".join(RU_LETTERS.findall(txt))
    
    # Приводим к нормальной форме и удаляем неинформативные слова длиной менее 2-х букв.
    # Также удаляем предлоги, сюозы, частицы, междометия
    words_filtered = []
    for word in txt.split():
        if len(word) > 2:
            words_filtered.append(word)
#             m = morph.parse(word)[0]
#             if m.tag.POS not in ('PREP', 'CONJ', 'PRCL', 'INTJ'):
#                 words_filtered.append(m.normal_form)
    return ' '.join(words_filtered)


# Приводим все слова к нормальной форме. Удаляем неинформативные части речи: предлоги, союзы,частицы, междометия.
def format_norm(txt: str) -> str:
    words = []
    for word in txt.split():
        m = morph.parse(word)[0]
        if m.tag.POS not in ('PREP', 'CONJ', 'PRCL', 'INTJ'):
            words.append(m.normal_form)
    return ' '.join(words)
    

In [5]:
# Подготовка данных для заданного датасета.

def format_df_inplace(df: pd.DataFrame) -> None:
    # "Причёсываем" Текст Сообщения.
    df['Текст Сообщения'] = df['Текст Сообщения'].apply(format_text)
    # Исправляем опечатки.
    df['Текст Сообщения'] = df['Текст Сообщения'].apply(spell)
    # Приводим к нормальной форме
    df['Текст Сообщения'] = df['Текст Сообщения'].apply(format_norm)
    
    # Тематику преобразовываем в набор токенов.
    df['Тематика'] = df['Тематика'].apply(format_text)
    # Нормализованные слова помещаем в отдельное поле, так как исходная Тематика нам пригодится
    df['Тематика Норм'] = df['Тематика'].apply(format_norm)
    
    # Ответственное лицо разбиваем на типы: Администрация, Комиссия, Другое
    df['Тип ответсвенного'] = df['Ответственное лицо'].apply(lambda x: 'Администрация' if x[:13] == 'Администрация' 
                                                             else 'Комитет' if x[:7] == 'Комитет' else 'Другое')

In [9]:
# # Еще раз читаем исходные данные на случай, если в ходе экспериментов в них уже что-то было поломано

# df_train = pd.read_csv('train_dataset_train.csv')
# df_test = pd.read_csv('test_dataset_test.csv')

# format_df_inplace(df_train)
# format_df_inplace(df_test)

# # Так как подготовка знимает много времени, сохраним результаты чтобы в следующий раз сразу загрузить уже подготовленные данные.
# df_train.to_csv('train_dataset_prepared.csv')
# df_test.to_csv('test_dataset_prepared.csv')

In [7]:
df_train = pd.read_csv('train_dataset_prepared.csv', index_col=0)
df_test = pd.read_csv('test_dataset_prepared.csv', index_col=0)


In [8]:
df_train.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория,Тематика Норм,Тип ответсвенного
0,2246,помочь начальник льговский рэс реагировать жал...,нарушения связанные содержанием электросети ка...,Администрация Льговского района,3,нарушение связанный содержание электросеть кач...,Администрация
1,380,фасад дом адрес урицкий проходить труба газовы...,аварийные деревья,Администрация города Курска,3,аварийный дерево,Администрация
2,2240,агрессивный собака радуга там стая подрасти ще...,безнадзорные животные,Администрация города Курска,1,безнадзорный животное,Администрация
3,596,пересечение улица сосновый береговой завалить ...,нескошенная сорная растительность местах общег...,Комитет дорожного хозяйства города Курска,3,нескошенный сорный растительность место общий ...,Комитет
4,1797,рядом дом улица светлый придомовый территория ...,аварийные деревья,Комитет городского хозяйства города Курска,3,аварийный дерево,Комитет


In [181]:
df_test.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Тематика Норм,Тип ответсвенного
0,843,улица мир быть заменить наружное освещение зам...,неработающее наружное освещение,Администрация Курчатовского района,неработающий наружное освещение,Администрация
1,1422,уже второй неделя гореть уличный освещение,неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...,неработающий наружное освещение,Комитет
2,2782,работать освещение двор дом энергетик световой...,неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...,неработающий наружное освещение,Комитет
3,2704,покос сорный растительность газон тротуар прое...,неудовлетворительная уборка улиц тротуаров,Администрация Центрального округа города Курска,неудовлетворительный уборка улица тротуар,Администрация
4,1,просить принять мера водитель маршрут пос севе...,неудовлетворительный внешний вид поведение вод...,Администрация города Курска,неудовлетворительный внешний вид поведение вод...,Администрация


## Оценка влияния полей на категорию обращения

Убедимся, что одной тематике соответствует одна и та же категория

In [10]:
df_train_grp_theme = df_train.groupby('Тематика')

In [11]:
df_train_grp_theme = df_train_grp_theme['Категория'].agg([('cat_min', 'min'), ('cat_max', 'max'), ('theme_count', 'count')])

In [12]:
np.sum(df_train_grp_theme.cat_min != df_train_grp_theme.cat_max)

0

В то же время можно заметить наличие большого количества тем, присутствующих только в одной записи.

In [13]:
df_train_grp_theme.theme_count.sort_values(axis=0, ascending=True).head(30)

Тематика
нарушения связанные обеспечением детским питанием                                                                                1
нарушение графика движения при осуществлении внутриобластных междугородных перевозок                                             1
нарушение графика движения при осуществлении муниципальных пригородных перевозок                                                 1
некачественное предоставление услуг почтовой связи                                                                               1
пешеходные переходы дорогах регионального межмуниципального значения                                                             1
неудовлетворительный внешний вид поведение водителя при осуществлении внутриобластных междугородных перевозок                    1
неудовлетворительный внешний вид поведение водителя при осуществлении муниципальных пригородных перевозок                        1
ошибка отображения данных услугах организациях ведомствах портале госуслуг

Это означает, что уже при кросс-валидации мы столкнемся с трудностью при попытке определить категорию по теме: 
подходящей темы в обучающем датасете может просто не оказаться.
Вместо этого мы будем искать не точно такую же тему, а похожую.
Но и в этом случае можно столкнуться с ситуацией, когда похожей методики не существует. 
Поэтому имеет смысл посмотреть и на другие поля.

In [34]:
pd.crosstab(df_train['Тип ответсвенного'], df_train['Категория'])

Категория,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Тип ответсвенного,,,,,,,,,,,,,,,,,
Администрация,280,11,1,369,0,9,5,13,53,0,12,11,0,6,1,2,76
Другое,3,0,0,242,2,0,0,0,19,0,1,1,1,1,0,0,3
Комитет,195,14,2,343,106,3,5,14,67,5,35,7,0,4,3,5,70


In [ ]:
# Построим словарь соответствия Тематика - Категория

In [10]:
# Строим словарь соответствий Тематика - Категория для заданного датасета

def create_theme_cat_map(df: pd.DataFrame) -> Dict:
    theme_cat = dict()
    for index, row in df.iterrows():
        if row['Тематика'] not in theme_cat:
            theme_cat[row['Тематика']] = row['Категория']
    return theme_cat

In [17]:
# Строим словарь соответствий "Множество слов из поля fld_name - Категория" для заданного датасета.

def create_theme_set_cat_map(df: pd.DataFrame, fld_name: str) -> Dict:
    theme_cat = dict()
    for index, row in df.iterrows():
        if row[fld_name] not in theme_cat:
            # theme_cat[frozenset(filter(lambda x: len(x) > 2, row['Тематика'].split()))] = row['Категория']
            theme_cat[frozenset(row[fld_name].split())] = row['Категория']
    return theme_cat

In [183]:
# df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train, df_train['Категория'], test_size=0.3)

In [17]:
# df_train_oper.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
678,1139,сегодня перод появилась вонь грибной радуги пр...,неприятные запахи,Комитет природных ресурсов Курской области,16
1468,2810,отсутствие люка люк сломан колодца пересечении...,отсутствие люков коммуникационных колодцах,Комитет жилищно-коммунального хозяйства города...,3
339,1580,спилите клен вокзальная курске трамвай едит ег...,аварийные деревья,Комитет городского хозяйства города Курска,3
1311,428,добрый день прошу взять особый контроль исполн...,ямы выбоины внутридворовых проездах тротуарах ...,Администрация города Курска,3
956,2596,добрый день засохло дерево года повреждены кор...,аварийные деревья,Комитет городского хозяйства города Курска,3


In [18]:
# df_test_oper.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
1316,364,как видно фото один изоляторов оторвался столб...,нарушения связанные содержанием электросети ка...,Администрация города Льгов,3
603,1693,добрый день данном портале уже озвучивалась да...,необходимо строительство тротуара,Комитет дорожного хозяйства города Курска,3
223,2034,хуторская дом разрушены ступени лестницы около...,несвоевременный некачественный текущий ремонт ...,Государственная жилищная инспекция Курской обл...,3
1994,587,здравствуйте щигровском районе зелёной роще на...,плохое материально техническое оснащение учреж...,Администрация Щигровского района,15
533,1195,яма проезжей части дороги обоянь луначарского ...,нарушение дорожного покрытия ямы дорогах грани...,Администрация г. Обояни,0


In [19]:
# theme_cat_map = create_theme_cat_map(df_train_oper)
# theme_set_cat_map = create_theme_set_cat_map(df_train_oper)

In [14]:
# Подбор категории на основе прямого сравнения множества слов в поле fld_name

def predict_simplest(df_train_data: pd.DataFrame, df_test_data: pd.DataFrame, fld_name: str) -> List[int]:
    theme_set_cat_map = create_theme_set_cat_map(df_train_data, fld_name)
    res_cat = []
    for index, row in df_test_data.iterrows():
        # theme_set = set(filter(lambda x: len(x) > 2, row['Тематика'].split()))
        theme_set = set(row[fld_name].split())
        cur_d = 0
        cur_cat = None
        for key in theme_set_cat_map:
            d = len(key.intersection(theme_set))
            if d > cur_d:
                cur_d = d
                cur_cat = theme_set_cat_map[key]
                # print(key.intersection(theme_set))
        # res_cat.append(cur_cat)
        # if (cur_d <= 3 and len(theme_set) <= 3) or cur_d > 3:
        if (cur_d == len(theme_set)) or cur_d > 3:
            res_cat.append(cur_cat)
        else:
            res_cat.append(None)
    return res_cat

In [15]:
# Проверка на кроссвалидации варианта с простейшим классификатором. Оценка количества неверно определившихся и не определившихся категорий.

def check_cv_simplest(fld_name: str):
    res_none = []
    res_correct = []
    res_incorrect = []
    for ind in range(50):
        df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train,
                                                                                  df_train['Категория'], 
                                                                                  test_size=0.3,
                                                                                  random_state=ind * 10)
        predict_y = np.array(predict_simplest(df_train_oper, df_test_oper, fld_name))
        res_none.append(np.sum(predict_y == None))
        res_correct.append(sum((predict_y == test_oper_y)[predict_y != None]))
        res_incorrect.append(sum((predict_y != test_oper_y)[predict_y != None]))
    return dict(res_none=res_none, res_correct=res_correct, res_incorrect=res_incorrect)

def print_stat_check_cv_simplest(r: Dict) -> None:
    print(f"Статистика None. max: {max(r['res_none'])}, min: {min(r['res_none'])}, mean: {np.mean(r['res_none'])}, median: {np.median(r['res_none'])}, sd: {np.std(r['res_none'])}")
    print(f"Статистика неверных ответов. max: {max(r['res_incorrect'])}, min: {min(r['res_incorrect'])}, mean: {np.mean(r['res_incorrect'])}, median: {np.median(r['res_incorrect'])}, sd: {np.std(r['res_incorrect'])}")
    print(f"Статистика верных ответов. max: {max(r['res_correct'])}, min: {min(r['res_correct'])}, mean: {np.mean(r['res_correct'])}, median: {np.median(r['res_correct'])}, sd: {np.std(r['res_correct'])}")


In [18]:
r = check_cv_simplest('Тематика')

In [27]:
print_stat_check_cv_simplest(r)

Статистика None. max: 18, min: 2, mean: 7.3, median: 7.0, sd: 3.395585369269929
Статистика неверных ответов. max: 3, min: 0, mean: 0.56, median: 0.0, sd: 0.6681317235396026
Статистика верных ответов. max: 598, min: 581, mean: 592.14, median: 592.5, sd: 3.5553340208762383


In [20]:
r_norm = check_cv_simplest('Тематика Норм')

In [28]:
print_stat_check_cv_simplest(r_norm)

Статистика None. max: 85, min: 59, mean: 69.22, median: 68.0, sd: 6.394654017224076
Статистика неверных ответов. max: 248, min: 187, mean: 212.9, median: 212.0, sd: 13.762630562505121
Статистика верных ответов. max: 345, min: 274, mean: 317.88, median: 315.5, sd: 14.710050985635638


Нормализация Тематики почти не повлияла на результат

Не ожидая каких-то серьёзных результатов, применим навскидку этот же подход к полю Текст Сообщения

In [30]:
r_txt_msg = check_cv_simplest('Текст Сообщения')

In [31]:
print_stat_check_cv_simplest(r_txt_msg)

Статистика None. max: 85, min: 59, mean: 69.22, median: 68.0, sd: 6.394654017224076
Статистика неверных ответов. max: 248, min: 187, mean: 212.9, median: 212.0, sd: 13.762630562505121
Статистика верных ответов. max: 345, min: 274, mean: 317.88, median: 315.5, sd: 14.710050985635638


#### Нормальная форма, без удаления предлогов, союзов и пр

Статистика None. max: 6, min: 0, mean: 0.68, median: 0.0, sd: 1.2560254774486066

Статистика неверных ответов. max: 10, min: 0, mean: 3.46, median: 3.0, sd: 2.22

Статистика верных ответов. max: 599, min: 584, mean: 595.86, median: 596.0, sd: 2.742334771686345


#### Нормальная форма с удалением предлогов, союзов и пр

Статистика None. max: 6, min: 0, mean: 0.68, median: 0.0, sd: 1.2560254774486066

Статистика неверных ответов. max: 10, min: 0, mean: 3.46, median: 3.0, sd: 2.22

Статистика верных ответов. max: 599, min: 584, mean: 595.86, median: 596.0, sd: 2.742334771686345

#### Без нормальной формы

Статистика None. max: 6, min: 0, mean: 1.28, median: 1.0, sd: 1.3570556362949897

Статистика неверных ответов. max: 8, min: 0, mean: 2.76, median: 3.0, sd: 1.8062115047801017

Статистика верных ответов. max: 599, min: 586, mean: 595.96, median: 596.0, sd: 2.3491274976041634


#### Без нормальной формы
        if (cur_d <= 3 and len(theme_set) <= 3) or cur_d > 3:

Статистика None. max: 17, min: 1, mean: 6.48, median: 6.0, sd: 3.3420951512486896

Статистика неверных ответов. max: 4, min: 0, mean: 0.64, median: 0.5, sd: 0.8187795796183488

Статистика верных ответов. max: 599, min: 582, mean: 592.88, median: 593.0, sd: 3.4562407323564717

#### Нормальная форма с удалением предлогов, союзов и пр
    if (cur_d <= 3 and len(theme_set) <= 3) or cur_d > 3:
    
Статистика None. max: 14, min: 1, mean: 6.1, median: 5.5, sd: 3.2264531609803355

Статистика неверных ответов. max: 4, min: 0, mean: 1.1, median: 1.0, sd: 1.0630145812734648

Статистика верных ответов. max: 599, min: 582, mean: 592.8, median: 593.0, sd: 3.63318042491699


#### Нормальная форма с удалением предлогов, союзов и пр
    if (cur_d == len(theme_set)) or cur_d > 3:
    
Статистика None. max: 15, min: 2, mean: 7.22, median: 7.0, sd: 3.245242671973854

Статистика неверных ответов. max: 4, min: 0, mean: 0.78, median: 1.0, sd: 0.9651942809610923

Статистика верных ответов. max: 598, min: 581, mean: 592.0, median: 592.0, sd: 3.671511950137164


#### Нормальная форма с удалением предлогов, союзов и пр
if (cur_d == len(theme_set)) or cur_d > 4:

Статистика None. max: 19, min: 3, mean: 9.78, median: 9.0, sd: 3.817276516051726

Статистика неверных ответов. max: 3, min: 0, mean: 0.22, median: 0.0, sd: 0.54

Статистика верных ответов. max: 597, min: 580, mean: 590.0, median: 591.0, sd: 3.9597979746446663


#### Без нормальной формы
if (cur_d == len(theme_set)) or cur_d > 3:

Статистика None. max: 18, min: 2, mean: 7.3, median: 7.0, sd: 3.395585369269929

Статистика неверных ответов. max: 3, min: 0, mean: 0.56, median: 0.0, sd: 0.6681317235396026

Статистика верных ответов. max: 598, min: 581, mean: 592.14, median: 592.5, sd: 3.5553340208762383


## Наивный Байес

$P(A|B) = $

In [33]:
# Универсальный (с точки зрения выбора текстовых полей для классификации) класс для выполнения классификации Наивным Байесом.

class NaiveBayes:
    def __init__(self, train_text_list: List[str], train_resp_type: List[str], train_cat_list: List[int]):
        if not (len(train_text_list) == len(train_cat_list) == len(train_resp_type)):
            raise Exception('Не совпадает длина входных списков')
            
        self.train_cnt = len(train_text_list)
        
        # Подсчитваем в скольких записях встречается каждое слово во всех категориях.
        # Сколько раз встречается каждое слово в каждой категории.
        # Сколько раз встречается каждый Тип ответсвенного лица в каждой категории
        # А также количество записей с каждой категорией

        self.word_cnt = defaultdict(int)
        self.word_cnt_by_cat = defaultdict(int)
        self.train_cnt_by_cat = defaultdict(int)
        self.resp_type_cnt_by_cat = defaultdict()
        for ind in range(len(train_text_list)):
            txt = train_text_list[ind]
            cat = train_cat_list[ind]
            self.train_cnt_by_cat[cat] += 1
            for word in set(txt.split()):
                self.word_cnt[word] += 1
                self.word_cnt_by_cat[(cat, word)] += 1
            self.resp_type_cnt_by_cat[(cat, train_resp_type[ind])] += 1
            
                
        self.resp_type_cnt_by_cat = dict()
        for ind in range(len(train_resp_type)):
            self.resp_type_cnt_by_cat[()]
        
        
    def get_word_cnt_in_cat(self, cat: int, word: str) -> int:
        return self.word_cnt_by_cat.get((cat, word)) or 1

    def get_word_cnt_not_in_cat(self, cat: int, word: str) -> int:
        return (self.train_cnt_by_cat[cat] - self.word_cnt_by_cat.get((cat, word), 0)) or 1
    
    def get_resp_type_cnt_in_cat(self, cat: int, resp_type: str) -> int:
        return self.resp_type_cnt_by_cat.get((cat, resp_type)) or 1
    
    def get_resp_type_cnt_not_in_cat(self, cat: int, resp_type: str) -> int:
        return (self.train_cnt_by_cat[cat] - self.resp_type_cnt_by_cat.get((cat, resp_type), 0)) or 1
    
    def get_cat_probs(self, txt: str, resp_type: str) -> List[float]:
        chance = [0] * 17
        word_set = set(txt.split())
        for cat in range(17):
            if self.train_cnt_by_cat[cat] < 32:
                chance[cat] = 0
            else:
                chance[cat] = self.train_cnt_by_cat[cat] / self.train_cnt
                for word in self.word_cnt:
                    chance[cat] *= (self.get_word_cnt_in_cat(cat, word) 
                                   if word in word_set else self.get_word_cnt_not_in_cat(cat, word)) / self.train_cnt_by_cat[cat]
                for 
                chance[cat] *= (self.get_resp_type_cnt_in_cat(cat, resp_type) 
                                if 
        sum_chance = sum(chance)
        return [ch / sum_chance for ch in chance]

Проверим, что получилось

In [35]:
df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train, df_train['Категория'], test_size=0.3, random_state=42)

In [36]:
bayes = NaiveBayes(df_train_oper['Текст Сообщения'].values, df_train_oper['Категория'].values)
b_pred_y = [np.argmax(bayes.get_cat_probs(txt)) for txt in df_test_oper['Текст Сообщения']]

In [38]:
sum(test_oper_y != b_pred_y), sum(test_oper_y == b_pred_y)

(360, 240)

In [39]:
bayes = NaiveBayes(df_train_oper['Тематика Норм'].values, df_train_oper['Категория'].values)
b_pred_y = [np.argmax(bayes.get_cat_probs(txt)) for txt in df_test_oper['Тематика Норм']]

In [40]:
sum(test_oper_y != b_pred_y), sum(test_oper_y == b_pred_y)

(53, 547)

In [46]:
# Проверка на кроссвалидации варианта с наивным Байесом. Оценка количества верно и неверно определившихся.

def check_cv_bayes(fld_name: str) -> Dict:
    res_correct = []
    res_incorrect = []
    for ind in range(50):
        df_train_oper, df_test_oper, train_oper_y, test_oper_y = train_test_split(df_train,
                                                                                  df_train['Категория'], 
                                                                                  test_size=0.3,
                                                                                  random_state=ind * 10)

        bayes = NaiveBayes(df_train_oper[fld_name].values, df_train_oper['Категория'].values)
        b_pred_y = [np.argmax(bayes.get_cat_probs(txt)) for txt in df_test_oper[fld_name]]

        res_correct.append(sum((b_pred_y == test_oper_y)))
        res_incorrect.append(sum((b_pred_y != test_oper_y)))
    return dict(res_correct=res_correct, res_incorrect=res_incorrect)

def print_stat_check_cv_bayes(r: Dict) -> None:
    print(f"Статистика неверных ответов. max: {max(r['res_incorrect'])}, min: {min(r['res_incorrect'])}, mean: {np.mean(r['res_incorrect'])}, median: {np.median(r['res_incorrect'])}, sd: {np.std(r['res_incorrect'])}")
    print(f"Статистика верных ответов. max: {max(r['res_correct'])}, min: {min(r['res_correct'])}, mean: {np.mean(r['res_correct'])}, median: {np.median(r['res_correct'])}, sd: {np.std(r['res_correct'])}")

In [49]:
r_cv_b = check_cv_bayes('Тематика')

In [50]:
print_stat_check_cv_bayes(r_cv_b)

Статистика неверных ответов. max: 82, min: 41, mean: 58.98, median: 57.0, sd: 10.15182742170098
Статистика верных ответов. max: 559, min: 518, mean: 541.02, median: 543.0, sd: 10.151827421700983


In [51]:
r_cv_b_norm = check_cv_bayes('Тематика Норм')

In [52]:
print_stat_check_cv_bayes(r_cv_b_norm)

Статистика неверных ответов. max: 80, min: 41, mean: 53.94, median: 50.0, sd: 9.577912089803288
Статистика верных ответов. max: 559, min: 520, mean: 546.06, median: 550.0, sd: 9.57791208980329


In [54]:
r_cv_b_msg = check_cv_bayes('Текст Сообщения')

In [55]:
print_stat_check_cv_bayes(r_cv_b_msg)

Статистика неверных ответов. max: 470, min: 337, mean: 388.06, median: 392.5, sd: 27.33891731579727
Статистика верных ответов. max: 263, min: 130, mean: 211.94, median: 207.5, sd: 27.33891731579727


In [55]:
df_train.shape[0]

(2000, 5)

In [56]:
random.randrange(0, df_train.shape[0])

1309

In [54]:
df_train.iloc[[random.randrange(0, df_train.shape[0]) for _ in range(df_train.shape[0])]]

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
1,380,фасад дом адрес урицкий проходить труба газовы...,аварийный дерево,Администрация города Курска,3
2,2240,агресивный собака радуга там стая подрасти щен...,безнадзорный животное,Администрация города Курска,1
3,596,пересечение улица сосновский береговой завалит...,нескошенный сорный растительность место общий ...,Комитет дорожного хозяйства города Курска,3


In [39]:
predict_y = np.array(simple_predict_1(df_test_oper))

In [40]:
sum((predict_y != test_oper_y)[predict_y != None])

0

In [41]:
print(np.sum(predict_y == None))
predict_y[predict_y == None] = 3

1


In [22]:
sum(predict_y != test_oper_y)

8

In [226]:
simple_predict_1(df_test_oper[predict_y != test_oper_y])

[5, 4, 8, 8]

In [227]:
predict_main = simple_predict_1(df_test)

In [228]:
np.sum(predict_main == None)

0

In [ ]:
format_df_inplace(df_train)

In [ ]:
df_train['Текст Сообщения'] = df_train['Текст Сообщения'].apply(format_text)
df_train['Текст Сообщения'] = df_train['Текст Сообщения'].apply(format_text)

In [ ]:
CLEAN_HTML = re.compile('<.*?>')
RU_LETTERS = re.compile('[а-я]+')
spell = Speller('ru')
morph = MorphAnalyzer()


In [56]:
def prepare_text(raw_text):
    # Убираем html-теги
    text = re.sub(CLEAN_HTML, '', raw_text)
    
    # Исправляем опечатки
    text = spell(text)
    
    # Приводим к нижнему регистру
    text = text.lower()
    
    # Оставляем только слова из русских букв
    text = " ".join(RU_LETTERS.findall(text))
    
    # Выполняем лемматизацию, оставляем только существительные и глаголы, 
    # полагая что основная смысловая нагрузка содержится в словах этих частей речи
    words_filtered = []
    for word in text.split():
        if len(word) > 2:
            m = morph.parse(word)[0]
            if m.tag.POS in ('NOUN', 'VERB', 'INFN'):
                words_filtered.append(m.normal_form)    
    return ' '.join(words_filtered)

In [2]:
df_train_orig = pd.read_csv('train_dataset_train.csv')
pd.read_csv('test_dataset_test.csv')

In [50]:
# Добавляем столбец с обработанным текстом сообщения

df_train_orig['TextPrepared'] = ''
for ind in range(len(df_train_orig['Текст Сообщения'].values)):
    df_train_orig['TextPrepared'].values[ind] = prepare_text(df_train_orig['Текст Сообщения'].values[ind])

In [52]:
df_train_orig.to_csv('train_prepared.csv')

In [54]:
set(df_train_orig['Категория'])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [3]:
df_test_orig = pd.read_csv('test_dataset_test.csv')

In [5]:
theme_train = set(df_train_orig['Тематика'])
theme_test = set(df_test_orig['Тематика'])

In [8]:
theme_test.difference(theme_train)

{'Дорожная разметка на дорогах регионального и межмуниципального значения',
 'Меры поддержки в условиях неблагоприятной эпидемиологической ситуации в сфере ЖКХ',
 'Меры поддержки в условиях неблагоприятной эпидемиологической ситуации в сфере занятости населения для частных лиц',
 'Необходимо строительство детской (спортивной) площадки в районе частного сектора',
 'Несоответствие ценника цене товара',
 'Несправедливое распределение мест в дошкольные учреждения',
 'Неудовлетворительное материально-техническое обеспечение учреждения дополнительного образования',
 'Отсутствие в населённом пункте сотовой связи',
 'Отсутствие твёрдого дорожного покрытия на дорогах регионального и межмуниципального значения',
 'Очистка  от снега и наледи  дорог в микрорайонах ИЖС',
 'Плохое материально-техническое оснащение учреждений культуры и библиотек',
 'Проблемы с обеспечением питанием в школах'}

In [9]:
train_words = []
for item in theme_train:
    train_words.append(set([w for w in item.split() if len(w) > 2]))

In [11]:
test_words = []
for item in theme_test:
    test_words.append(set([w for w in item.split() if len(w) > 2]))

In [20]:
for item in test_words:
    for pp in train_words:
        if len(pp.intersection(item)) >= 1:
            break
    else:
        print(item)
        

{'цене', 'ценника', 'Несоответствие', 'товара'}


In [12]:
test_words

[{'Сертификат', 'вакцинированного'},
 {'Пешеходные',
  'дорогах',
  'значения',
  'межмуниципального',
  'переходы',
  'регионального'},
 {'Парковки',
  'городских',
  'границах',
  'дорогах',
  'округов',
  'поселений',
  'сельских'},
 {'(некачественный)',
  'Несвоевременный',
  'дома',
  'многоквартирного',
  'ремонт',
  'текущий'},
 {'Неудовлетворительное', 'качество', 'оказания', 'товара,', 'услуги'},
 {'Безопасная',
  'городских',
  'границах',
  'дорога',
  'дорогах',
  'округов',
  'поселений',
  'сельских',
  'школу'},
 {'Отсутствие', 'колодцах', 'коммуникационных', 'люков'},
 {'Неудовлетворительная',
  'доме',
  'мест',
  'многоквартирном',
  'общего',
  'освещение',
  'пользования',
  'уборка'},
 {'Неисправный', 'доме', 'лифт', 'многоквартирном'},
 {'Проблемы', 'обеспечением', 'питанием', 'школах'},
 {'Неудовлетворительное',
  'здравоохранения',
  'материально-техническое',
  'состояние',
  'учреждений'},
 {'Несвоевременное',
  'благоустройства',
  'восстановление',
  'наруше

In [36]:
morph.parse('стучали')[0]

Parse(word='стучали', tag=OpencorporaTag('VERB,impf,intr plur,past,indc'), normal_form='стучать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стучали', 564, 10),))

In [4]:
df_train_orig = pd.read_csv('train_dataset_train.csv')

In [5]:
df_train_orig.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
0,2246,Помогите начальник Льговского рэс не реагирует...,"Нарушения, связанные с содержанием электросети...",Администрация Льговского района,3
1,380,<p>По фасаду дома по адресу ул. Урицкого 22 пр...,Аварийные деревья,Администрация города Курска,3
2,2240,Агресивные собаки. На радуге там стая из подро...,Безнадзорные животные,Администрация города Курска,1
3,596,<p>На пересечении &nbsp;улиц Сосновская и Бере...,Нескошенная сорная растительность в местах общ...,Комитет дорожного хозяйства города Курска,3
4,1797,<p style=`text-align:justify;`><span style=`ba...,Аварийные деревья,Комитет городского хозяйства города Курска,3


In [45]:
prepare_text(df_train_orig['Текст Сообщения'][5])

'день компания аврора убирать территория адрес курск звонить компания результат снегопад ситуация измениться просить мера аврора обязательство год взять житель дом'

In [24]:
set(df_train_orig['Тематика'])

{'«Замороженная» стройка',
 'Аварийное жильё',
 'Аварийные деревья',
 'Безнадзорные животные',
 'Безопасная дорога в школу на дорогах в границах городских округов и сельских поселений',
 'Вакцинация от COVID-19',
 'Водоотведение',
 'Вопросы оказания помощи беженцам',
 'Вопросы предоставления государственных и муниципальных услуг',
 'Длительное неисполнение заявок управляющей компанией',
 'Дневник самонаблюдения',
 'Дорожная разметка на дорогах в границах городских округов и сельских поселений',
 'Дорожные знаки на дорогах в границах городских округов и сельских поселений',
 'Дорожные знаки на дорогах в микрорайонах ИЖС',
 'Дорожные знаки на дорогах регионального и межмуниципального значения',
 'Загрязнение водных объектов',
 'Загрязнение территории прилегающей к строительному объекту',
 'Искусственные неровности на дорогах в границах городских округов и сельских поселений',
 'Искусственные неровности на дорогах регионального и межмуниципального значения',
 'Ливневая канализация',
 'Мер

In [23]:
len(set(df_train_orig['Тематика']))

161

In [ ]:
len(set(df_train_orig['Ответственное лицо']))

In [25]:
set(df_train_orig['Ответственное лицо'])

{'Администрация Беловского района',
 'Администрация Большесолдатского района',
 'Администрация Глушковского района',
 'Администрация Горшеченского района',
 'Администрация Железногорского района',
 'Администрация Железнодорожного округа города Курска',
 'Администрация Золотухинского района',
 'Администрация Касторенского района',
 'Администрация Конышевского района',
 'Администрация Кореневского района',
 'Администрация Курского района',
 'Администрация Курчатовского района',
 'Администрация Льговского района',
 'Администрация Мантуровского района',
 'Администрация Медвенского района',
 'Администрация Обоянского района',
 'Администрация Октябрьского района',
 'Администрация Поныровского района',
 'Администрация Пристенского района',
 'Администрация Рыльского района',
 'Администрация Сеймского округа города Курска',
 'Администрация Советского района',
 'Администрация Солнцевского района',
 'Администрация Суджанского района',
 'Администрация Тимского района',
 'Администрация Фатежского р

In [ ]:
df_train_orig.head()

In [26]:

spell = Speller('ru')
text = 'Проверкка текста на ашибки.'
print(spell(text))

dictionary for this language not found, downloading...
__________________________________________________
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
done!
Проверка текста на ошибки.


In [ ]:
import language_tool_python

tool = language_tool_python.LanguageTool('ru-RU')
text = 'Проверкка текста на ашибки.'
matches = tool.check(text)
print(matches)

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/myuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/myuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
russian_stopwords = stopwords.words("russian")


In [8]:
russian_stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [19]:
text = '''Добрый день! После регулировки светофоров на данном участке светофор на главной дороге стал работать некорректно: когда у водителей горит жёлтый сигнал, у пешеходов загорается зелёный, происходит одновременное движение автомобилей и пешеходов. Рядом находится Дворец пионеров, в котором занимаются тысячи детей. Неоднократно на моих глазах чуть не сбивали людей. Прошу принять меры, спасибо!</p>"'''

In [22]:
text = '''Агресивные собаки. На радуге там стая из подросших щенков и зврослых собак. Они на Ребенка  бросилась.  Соьаки бросаются пытаются за ноги  укусить. Разговор с охранниками не дал результатов'''

In [23]:
print(spell(text))

Агрессивные собаки. На радуге там стая из подросших щенков и взрослых собак. Они на Ребенка  бросилась.  Собаки бросаются пытаются за ноги  укусить. Разговор с охранниками не дал результатов


In [14]:
text_tokens = word_tokenize(text)

In [15]:
text_tokens

['Добрый',
 'день',
 '!',
 'После',
 'регулировки',
 'светофоров',
 'на',
 'данном',
 'участке',
 'светофор',
 'на',
 'главной',
 'дороге',
 'стал',
 'работать',
 'некорректно',
 ':',
 'когда',
 'у',
 'водителей',
 'горит',
 'жёлтый',
 'сигнал',
 ',',
 'у',
 'пешеходов',
 'загорается',
 'зелёный',
 ',',
 'происходит',
 'одновременное',
 'движение',
 'автомобилей',
 'и',
 'пешеходов',
 '.',
 'Рядом',
 'находится',
 'Дворец',
 'пионеров',
 ',',
 'в',
 'котором',
 'занимаются',
 'тысячи',
 'детей',
 '.',
 'Неоднократно',
 'на',
 'моих',
 'глазах',
 'чуть',
 'не',
 'сбивали',
 'людей',
 '.',
 'Прошу',
 'принять',
 'меры',
 ',',
 'спасибо',
 '!',
 '<',
 '/p',
 '>',
 "''"]